<a href="https://colab.research.google.com/github/SharShubeMayJune/Fake-news-projects/blob/main/Fake_News01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About data set
1 ID: Unique ID for a news article
2 Title: the tittle of a news article
3 Author: author of the new article
4 Text: the text of the aritcle; could be incomplete
5 Label: a label that marks whether the news article is real or fake

In [117]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [118]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [119]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [121]:
news_dataset = pd.read_csv('/content/train001.csv', encoding='ISO-8859-1')


<ipython-input-121-7afbb16ed93c>:1: DtypeWarning: Columns (49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  news_dataset = pd.read_csv('/content/train001.csv', encoding='ISO-8859-1')


In [125]:
news_dataset.shape

(4836, 686)

In [127]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id                 0
title            418
author           898
text             604
label            696
                ... 
Unnamed: 681    4836
Unnamed: 682    4836
Unnamed: 683    4836
Unnamed: 684    4836
Unnamed: 685    4836
Length: 686, dtype: int64

In [129]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [130]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

<ipython-input-130-5db6adfd5692>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']


In [131]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
4831    Alex Ansary Hillary Reboots âVast Right-Wing...
4832    Chad Bray Bank of England Cuts Interest Rate t...
4833       Steve Sailer NYT: Whites Must Know Their Place
4834    Michael Tennant Brits Fined and Prosecuted for...
4835    Cartel Chronicles Raging Cartel War Kills 20 i...
Name: content, Length: 4836, dtype: object


In [132]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [133]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didnât Even See Comeyâs...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
4831  3955  Hillary Reboots âVast Right-Wing Conspiracyâ...   
4832  3956  Bank of England Cuts Interest Rate to Historic...   
4833  3957                  NYT: Whites Must Know Their Place   
4834  3958  Brits Fined and Prosecuted for Weeds, Weeping,...   
4835  3959  Raging Cartel War Kills 20 in Mexico at Texas ...   

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didnât Even See Comeyâs...   
1        Daniel J. Flynn  Ever get the feeling your life circ

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [134]:
port_stem = PorterStemmer()

In [135]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [136]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [137]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
4831    alex ansari hillari reboot vast right wing con...
4832    chad bray bank england cut interest rate histo...
4833               steve sailer nyt white must know place
4834    michael tennant brit fine prosecut weed weep w...
4835    cartel chronicl rage cartel war kill mexico te...
Name: content, Length: 4836, dtype: object


In [138]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [139]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'steve sailer nyt white must know place'
 'michael tennant brit fine prosecut weed weep whippoorwil'
 'cartel chronicl rage cartel war kill mexico texa border']


In [140]:
print(Y)

['1' '0' '1' ... '1' '1' '0']


In [141]:
Y.shape

(4836,)

In [142]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [143]:
print(X)

  (0, 8112)	0.2867139643734818
  (0, 6944)	0.2397811917874691
  (0, 4656)	0.35822632235594204
  (0, 4514)	0.2955843285479585
  (0, 4057)	0.24976123005602796
  (0, 3690)	0.22245970184357464
  (0, 2602)	0.22884770508843144
  (0, 1996)	0.26764855675718513
  (0, 1893)	0.35822632235594204
  (0, 1550)	0.25451279481841704
  (0, 1309)	0.3755478122998365
  (0, 161)	0.2731096048592869
  (1, 8615)	0.29316675105578194
  (1, 3594)	0.19111683713747385
  (1, 2910)	0.72139305836862
  (1, 1878)	0.2714270182346986
  (1, 1468)	0.1944107813377604
  (1, 1185)	0.37471022430496975
  (1, 1013)	0.15985056949635001
  (1, 801)	0.2822968846452403
  (2, 8077)	0.415270727419403
  (2, 4999)	0.4889950023259024
  (2, 3152)	0.3322756354670344
  (2, 2844)	0.38404172287853394
  (2, 1640)	0.4696896000344177
  :	:
  (4832, 1001)	0.3328424682831444
  (4832, 626)	0.26399109540216964
  (4833, 8542)	0.3152046058127953
  (4833, 7507)	0.38561620427427284
  (4833, 6770)	0.44085088343400264
  (4833, 5904)	0.35772438965854614
  (48

In [151]:
from collections import defaultdict

# Create a dictionary to store indices of each class
class_indices = defaultdict(list)
for i, label in enumerate(Y):
    class_indices[label].append(i)

# Initialize lists to store indices for the training and testing sets
train_indices = []
test_indices = []

# Split each class into training and testing sets
for indices in class_indices.values():
    n = len(indices)
    train_size = int(n * 0.8)  # 80% for training, adjust as needed
    train_indices.extend(indices[:train_size])
    test_indices.extend(indices[train_size:])

# Split the features and target variable based on the indices
X_train, X_test = X[train_indices], X[test_indices]
Y_train, Y_test = Y[train_indices], Y[test_indices]


Training the Model: Logistic Regression

In [152]:
model = LogisticRegression()

In [153]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation


accuracy score

In [154]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [155]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9156106767322728


In [156]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [158]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.676131322094055


Predictive System

In [161]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['1']
The news is Fake


In [163]:
print(Y_test[1])

1
